In [0]:
# secrets = dbutils.secrets.list("dbc-training-scope")
# for secret in secrets:
#     print(secret.key)
#

In [0]:
# for letter in dbutils.secrets.get("dbc-training-scope","MySqlChampionsLeaguePassword"):
#    print(letter)

In [0]:
import requests
import json
from pyspark.sql.functions import current_timestamp, lit
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType, MapType, ArrayType

# Widget: Source system
dbutils.widgets.dropdown("src_system", "REST API", ["REST API", "MySQL"], "Select Source System")
src_system = dbutils.widgets.get("src_system")

def get_secrets(scope, username_key, password_key):
    """
    Retrieve the username and password from Databricks secrets scope.
    """
    username = dbutils.secrets.get(scope, username_key)
    password = dbutils.secrets.get(scope, password_key)
    return username, password

def get_token(username, password):
    """
    Authenticate and retrieve the access token using the provided username and password.
    """
    login_url = "http://detrainingbg.adastragrp.com:443/login"
    payload = {
        "username": username,
        "password": password
    }
    headers = {
        "Content-Type": "application/json"
    }

    response = requests.post(login_url, headers=headers, data=json.dumps(payload))

    if response.status_code == 200:
        bearer_token = response.json().get("access_token")
        return bearer_token
    else:
        print(f"Login failed. Status code: {response.status_code}")
        print(response.text)
        return None

def fetch_teams(bearer_token):
    """
    Fetch teams using the provided bearer token.
    """
    teams_url = "http://detrainingbg.adastragrp.com:443/teams"

    headers = {
        "Authorization": f"Bearer {bearer_token}"
    }

    all_teams = []
    page_cursor = None

    while True:
        params = {"page_cursor": page_cursor} if page_cursor else {}
        response = requests.get(teams_url, headers=headers, params=params)

        if response.status_code != 200:
            print(f"Failed to retrieve teams. Status code: {response.status_code}")
            print(response.text)
            break
        
        data = response.json()

        # Collect teams from the current page
        teams = data.get("teams") 
        all_teams.extend(teams)

        # Get next page cursor 
        page_cursor = data.get("page_cursor")

        if not page_cursor:
            break  # no more pages

    teams_schema = StructType([
    StructField("id_team", IntegerType(), True),
    StructField("country", StringType(), True),
    StructField("team", StringType(), True),
    StructField("logo", StringType(), True)
    ])
    teams_df = spark.createDataFrame(all_teams, schema=teams_schema)

    save_to_bronze(teams_df, "teams")

    return all_teams    

def fetch_players(bearer_token, teams):
    """
    Fetch players using the provided bearer token and iterate through the teams to get the players from that team.
    """
    players_url = "http://detrainingbg.adastragrp.com:443/players"
    headers = {"Authorization": f"Bearer {bearer_token}"}
    all_players = []

    for team in teams:
        team_id = team.get("id_team")
        if not team_id:
            continue

        params = {"id_team": team_id}
        response = requests.get(players_url, headers=headers, params=params)

        if response.status_code != 200:
            print(f"Failed to retrieve players for team {team_id}")
            continue

        players = response.json()
        for p in players:
            p["id_team"] = team_id
            all_players.append(p)

    # Define Spark schema for players
    player_schema = StructType([
        StructField("id_player", IntegerType(), True),
        StructField("player_name", StringType(), True),
        StructField("nationality", StringType(), True),
        StructField("field_position", StringType(), True),
        StructField("position", StringType(), True),
        StructField("player_image", StringType(), True),
        StructField("bio", MapType(StringType(), StringType()), True),
        StructField("match_statistics", ArrayType(MapType(StringType(), StringType())), True),
        StructField("id_team", IntegerType(), True)
    ])

    players_df = spark.createDataFrame(all_players, schema=player_schema)

    save_to_bronze(players_df, "players")

def fetch_mysql_data(username, password):
    tables = ["goals_data", "attempts_data", "attacking_data"]

    for table in tables:
        df = (spark.read
            .format("mysql")
            .option("dbtable", table)
            .option("host", "detrainingbg.adastragrp.com")
            .option("port", "3306")
            .option("database", "champions_league")
            .option("user", username)
            .option("password", password)
            .load()
        )

        save_to_bronze(df, table)

def save_to_bronze(df, table_name):
    """
    Save the data to the bronze tables.
    """

    df = df.withColumn("load_ts", current_timestamp()) \
                           .withColumn("src_system", lit(src_system))
    
    # Overwrite the table in the specified schema
    df.write.format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(f"bronze.training_io.{table_name}")

def main():

    # Get secrets
    APIusername, APIpassword = get_secrets("dbc-training-scope", "FastAPIChampionsLeagueUsername", "FastAPIChampionsLeaguePassword")
    MySQLusername, MySQLpassword = get_secrets("dbc-training-scope", "MySqlChampionsLeagueUser", "MySqlChampionsLeaguePassword")
    
    # Login and get bearer token
    bearer_token = get_token(APIusername, APIpassword)
    if bearer_token:
        # Fetch teams
        teams = fetch_teams(bearer_token)
        fetch_players(bearer_token, teams)
    
    fetch_mysql_data(MySQLusername, MySQLpassword)

if __name__ == "__main__":
    main()
